<a href="https://colab.research.google.com/github/DB9653/Bachelorarbeit/blob/main/HASOCOne_(TrainGE19_TestGE18).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CUDA and DRIVE

In [ ]:
## https://github.com/suman101112/hasoc-fire-2020/blob/main/HASOCOne_team_german_task_a.ipynb

import torch

if torch.cuda.is_available():    
    device = torch.device("cuda")
else:
    device = torch.device("cpu")



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
device


device(type='cuda')

# DATA

In [ ]:
import pandas as pd
import math
from sklearn import preprocessing
import csv

#file = "hasoc_2020_de_train_new_a.xlsx"
#file_test = "german_test_1509.csv"

# df_train = pd.read_csv("/content/german_dataset2019.tsv",sep="\t", converters={'tweet_id':int})

df_train = pd.read_csv("/content/germeval2019.training_subtask1_2_korrigiert.txt",sep="\t", encoding = "UTF-8", quoting=csv.QUOTE_NONE, names=("text", "task1", "task2"))

df_test = pd.read_csv("/content/germeval2018test.txt",sep='\t', encoding = "UTF-8", quoting=csv.QUOTE_NONE, names=("text", "task1", "task2"))



print(f"Len df_train: {len(df_train)}")
display(df_train)



Len df_train: 3995


,text,task1,task2
0,@jouwatch Hat die Polizei keine Kanone mehr ? ...,OFFENSE,ABUSE
1,@de_sputnik @Saudi Arabien habt ihr mal wieder...,OFFENSE,ABUSE
2,"Glaube ich nicht , die Bundesregierung so wie ...",OFFENSE,ABUSE
3,""" Doch schockierend viele Jugendliche wissen k...",OFFENSE,PROFANITY
4,Sein Charakter war ihm wichtiger anstatt als b...,OFFENSE,PROFANITY
...,...,...,...
3990,250 Menschen auf der Demonstration gegen das D...,OTHER,OTHER
3991,Erneut Massaker an Kurdische ZivilistInnen dur...,OTHER,OTHER
3992,Hunderte Refugees haben die Grenze zur spanisc...,OTHER,OTHER
3993,Heute ab 17:00 Uhr an der Alten Oper FFM: Kund...,OTHER,OTHER


In [ ]:
print(df_train.dtypes)
display(df_train)

text     object
task1    object
task2    object
dtype: object


,text,task1,task2
0,@jouwatch Hat die Polizei keine Kanone mehr ? ...,OFFENSE,ABUSE
1,@de_sputnik @Saudi Arabien habt ihr mal wieder...,OFFENSE,ABUSE
2,"Glaube ich nicht , die Bundesregierung so wie ...",OFFENSE,ABUSE
3,""" Doch schockierend viele Jugendliche wissen k...",OFFENSE,PROFANITY
4,Sein Charakter war ihm wichtiger anstatt als b...,OFFENSE,PROFANITY
...,...,...,...
3990,250 Menschen auf der Demonstration gegen das D...,OTHER,OTHER
3991,Erneut Massaker an Kurdische ZivilistInnen dur...,OTHER,OTHER
3992,Hunderte Refugees haben die Grenze zur spanisc...,OTHER,OTHER
3993,Heute ab 17:00 Uhr an der Alten Oper FFM: Kund...,OTHER,OTHER


In [ ]:
print(df_test.dtypes)
display(df_test)

text     object
task1    object
task2    object
dtype: object


,text,task1,task2
0,"Meine Mutter hat mir erzählt, dass mein Vater ...",OTHER,OTHER
1,@Tom174_ @davidbest95 Meine Reaktion; |LBR| Ni...,OTHER,OTHER
2,"#Merkel rollt dem Emir von #Katar, der islamis...",OTHER,OTHER
3,„Merle ist kein junges unschuldiges Mädchen“ K...,OTHER,OTHER
4,@umweltundaktiv Asylantenflut bringt eben nur ...,OFFENSE,ABUSE
...,...,...,...
3527,"@schlabonski tja, es können einen nicht alle L...",OTHER,OTHER
3528,"@RKnillmann @lawyerberlin @AfD Aha, der Islam ...",OFFENSE,ABUSE
3529,"@podilein Mannheim, weltoffen und kunterbunt. ...",OFFENSE,ABUSE
3530,@stephanweil was ist nun mit kostenlosen Kitas...,OTHER,OTHER


In [ ]:
labelsss = df_train['task1']
print(labelsss.value_counts())

OTHER      2708
OFFENSE    1287
Name: task1, dtype: int64


In [ ]:
labelsss = df_test['task1']
print(labelsss.value_counts())

OTHER      2330
OFFENSE    1202
Name: task1, dtype: int64


In [ ]:
total_sentences = list(df_train['text'].values)
total_labels = list(df_train['task1'].values)

test_sentences = list(df_test['text'].values)
test_labels = list(df_test['task1'].values)

# Cleaning, Transforming, Encoding

In [ ]:
def clean_text(sentences):
    for index,line in enumerate(sentences):
        if "\n" in line:
            sentences[index] = line.replace("\n","")
    return sentences
        
total_sentences = clean_text(total_sentences)
test_sentences = clean_text(test_sentences)

def clean_labels(labels):
    new_list= []
    for value in labels:
        new_list.append(value.strip())
    return new_list

total_labels = clean_labels(total_labels)
test_labels = clean_labels(test_labels)

le = preprocessing.LabelEncoder()
le.fit(total_labels)
encoded_labels = le.transform(total_labels)
encoded_test_labels = le.transform(test_labels)
print("encoded labels:", set(encoded_labels))

print(f"\nLen.: total_sentences: {len(total_sentences)}, encoded labels: {len(encoded_labels)}, test_sentences: {len(test_sentences)}, encoded_test_labels: {len(encoded_test_labels)}")

display(df_test)



encoded labels: {0, 1}

Len.: total_sentences: 3995, encoded labels: 3995, test_sentences: 3532, encoded_test_labels: 3532


,text,task1,task2
0,"Meine Mutter hat mir erzählt, dass mein Vater ...",OTHER,OTHER
1,@Tom174_ @davidbest95 Meine Reaktion; |LBR| Ni...,OTHER,OTHER
2,"#Merkel rollt dem Emir von #Katar, der islamis...",OTHER,OTHER
3,„Merle ist kein junges unschuldiges Mädchen“ K...,OTHER,OTHER
4,@umweltundaktiv Asylantenflut bringt eben nur ...,OFFENSE,ABUSE
...,...,...,...
3527,"@schlabonski tja, es können einen nicht alle L...",OTHER,OTHER
3528,"@RKnillmann @lawyerberlin @AfD Aha, der Islam ...",OFFENSE,ABUSE
3529,"@podilein Mannheim, weltoffen und kunterbunt. ...",OFFENSE,ABUSE
3530,@stephanweil was ist nun mit kostenlosen Kitas...,OTHER,OTHER


In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 28.5 MB/s 
     |████████████████████████████████| 6.6 MB 57.3 MB/s 
     |████████████████████████████████| 101 kB 14.3 MB/s 


In [ ]:
from transformers import BertTokenizer


tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

max_length = 0
for sentence in total_sentences:
    #print(sentence)
    length = len(tokenizer.tokenize(sentence))
    if length > max_length:
        max_length  = length
print("max token length is: ",max_length)
# max token length obtained is 50
# bert tokens are limited to 514 bytes.



max token length is:  361


In [ ]:
def encoder_generator(sentences,labels):
    
    sent_index = []
    input_ids = []
    attention_masks =[]

    for index,sent in enumerate(sentences):
        
        sent_index.append(index)
        
        encoded_dict = tokenizer.encode_plus(sent,
                                             add_special_tokens=True,

                                             #max_length geändert, da die "echte"
                                             # max_length zu lang ist
                                             max_length=512,

                                             #deprecation, padding geändert
                                             padding='max_length',

                                             truncation = True,
                                             return_attention_mask=True,
                                             return_tensors='pt')
        input_ids.append(encoded_dict['input_ids'])

        attention_masks.append(encoded_dict['attention_mask'])

    input_ids = torch.cat(input_ids,dim=0)
    attention_masks = torch.cat(attention_masks,dim=0)
    labels = torch.tensor(labels)
    sent_index = torch.tensor(sent_index)

    return sent_index,input_ids,attention_masks,labels

sent_index,input_ids,attention_masks,encoded_label_tensors = encoder_generator(total_sentences,encoded_labels)
test_sent_index,test_input_ids,test_attention_masks,encoded_test_label_tensors = encoder_generator(test_sentences,encoded_test_labels)
print('Original: ', total_sentences[0])
print('Token IDs:', input_ids[0])
#print(encoded_label_tensors)
#print(encoded_test_label_tensors)

Original:  @jouwatch Hat die Polizei keine Kanone mehr ? 20 mal in die Munition laufen lassen und Ruhe ist .
Token IDs: tensor([   101,    137,  12541, 104388,  38732,  48024,  10128,  42974,  14618,
         57528,  12926,  12471,    136,  10197,  15189,  10106,  10128,  49056,
         78362,  27207,  15559,  23098,  10130,    155,  18593,  10112,  10298,
           119,    102,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,


# Split into train, val and test

In [ ]:
from torch.utils.data import TensorDataset,random_split

dataset = TensorDataset(input_ids,attention_masks,encoded_label_tensors)
test_dataset = TensorDataset(test_sent_index,test_input_ids,test_attention_masks,encoded_test_label_tensors)

train_size = int(0.75*len(dataset))

val_size = len(dataset)-train_size

train_dataset,val_dataset = random_split(dataset,[train_size,val_size])

print('train data samples is {}'.format(len(train_dataset)))
print("valid data samples is {}".format(len(val_dataset)))
print("test data samples is {}".format(len(test_dataset)))

train data samples is 2996
valid data samples is 999
test data samples is 3532


In [ ]:
from torch.utils.data import DataLoader,RandomSampler,SequentialSampler

bs=8

train_data_loader = DataLoader(train_dataset,
                              sampler=RandomSampler(train_dataset),
                              batch_size=bs)
valid_data_loader = DataLoader(val_dataset,
                              sampler=SequentialSampler(val_dataset),
                              batch_size=bs)
test_data_loader = DataLoader(test_dataset,
                            sampler=SequentialSampler(test_dataset),
                            batch_size=bs)



# MODEL

In [ ]:
from transformers import BertForSequenceClassification, AdamW

model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased',
                                                     num_labels=len(le.classes_),
                                                     output_attentions=False,
                                                     output_hidden_states=False,
                                                     )
model.cuda()
#model.cpu()



Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

In [ ]:
# deprecation, AdamW zu torch.optim.AdamW
optimizer = torch.optim.AdamW(model.parameters(),lr=2e-5,eps=1e-8)

from transformers import get_linear_schedule_with_warmup

epochs=10
total_steps = len(train_data_loader) * epochs

scheduler = get_linear_schedule_with_warmup(optimizer,
                                           num_warmup_steps=0,
                                           num_training_steps=total_steps)



In [ ]:
import numpy as np

def predictions_labels(preds,labels):
    pred = np.argmax(preds,axis=1).flatten()
    label = labels.flatten()
    return pred,label



In [ ]:
import random
#import numpy as np
import time
from sklearn.metrics import classification_report,accuracy_score,f1_score

total_t0 = time.time()

seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)



In [ ]:
torch.FloatTensor.device

<attribute 'device' of 'torch._C._TensorBase' objects>

# Train and Evaluate

In [ ]:
def categorical_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """
    max_preds = preds.argmax(dim = 1, keepdim = True) # get the index of the max probability
    correct = max_preds.squeeze(1).eq(y)
    #return correct.sum() / torch.FloatTensor([y.shape[0]]).cuda()-1

    return correct.sum() / torch.FloatTensor([y.shape[0]]).cuda()
    #return correct.sum() 
def predictions_labels(preds,labels):
    pred = np.argmax(preds,axis=1).flatten()
    label = labels.flatten()
    return pred,label



In [ ]:
#print(torch.FloatTensor.parameters().device)
print(torch.FloatTensor.device)

print(next(model.parameters()).device)

<attribute 'device' of 'torch._C._TensorBase' objects>
cuda:0


In [ ]:
def train():
    total_train_loss = 0
    total_train_acc = 0
    
    model.train() # set model in train mode for batchnorm and dropout layers in bert model
    
    for step,batch in enumerate(train_data_loader):
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        model.zero_grad()
            
        #loss,logits = model(b_input_ids,
        #                    attention_mask = b_input_mask,
        #                    labels = b_labels.long()).values() 
                            #return_dict = False
                            #).values()


        output = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
        loss = output[0]
        logits = output[1]
            
        total_train_loss += loss.item()
        total_train_acc += categorical_accuracy(logits,b_labels).item()
            
        loss.backward()
            
        torch.nn.utils.clip_grad_norm_(model.parameters(),1.0)
            
        optimizer.step()
            
        scheduler.step() #go ahead and update the learning rate
            
    avg_train_loss = total_train_loss/len(train_data_loader)
    avg_train_acc = total_train_acc/len(train_data_loader)
    
    return avg_train_loss,avg_train_acc

In [ ]:
def evaluate():
    model.eval()
        
    total_eval_accuracy = 0
    total_eval_loss = 0
    number_of_eval_steps= 0
    
    all_true_labels = []
    all_pred_labels = []

    for batch in valid_data_loader:
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        with torch.no_grad():

            loss, logits = model(b_input_ids,
                                attention_mask= b_input_mask,
                                labels = b_labels.long()).values()
                                #return_dict = False
                                #).values()
            
            #answer_start_scores, answer_end_scores = model(**inputs).values()

            #outputs = model(**inputs)
            #answer_start_scores = outputs.start_logits
            #answer_end_scores = outputs.end_logits

        total_eval_loss += loss.item()

        logits = logits.detach().cpu().numpy()

        label_ids = b_labels.to('cpu').numpy()

        pred,true = predictions_labels(logits,label_ids)
        
        all_pred_labels.extend(pred)
        all_true_labels.extend(true)

    print(classification_report(all_pred_labels,all_true_labels))
    avg_val_accuracy = accuracy_score(all_pred_labels,all_true_labels)
    macro_f1_score = f1_score(all_pred_labels,all_true_labels,average='macro')
    
    avg_val_loss = total_eval_loss/len(valid_data_loader)

    print("accuracy = {0:.2f}".format(avg_val_accuracy))
    
    return avg_val_loss,avg_val_accuracy,macro_f1_score

In [ ]:
import time
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
epochs = 7

best_macro_f1 = float('0')

for epoch in range(epochs):
    
    start_time = time.time()
    train_loss,train_acc = train()
    valid_loss,valid_acc,macro_f1 = evaluate()
    
    end_time = time.time()
        
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
        
    if macro_f1 > best_macro_f1:
        best_macro_f1 = macro_f1
        torch.save(model,'HASOCOne_model_german_task_a_GermEval2019.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

              precision    recall  f1-score   support

           0       0.47      0.65      0.54       230
           1       0.88      0.78      0.83       769

    accuracy                           0.75       999
   macro avg       0.67      0.71      0.69       999
weighted avg       0.79      0.75      0.76       999

accuracy = 0.75
Epoch: 01 | Epoch Time: 5m 26s
	Train Loss: 0.620 | Train Acc: 67.63%
	 Val. Loss: 0.553 |  Val. Acc: 74.97%
              precision    recall  f1-score   support

           0       0.70      0.65      0.67       347
           1       0.82      0.86      0.84       652

    accuracy                           0.78       999
   macro avg       0.76      0.75      0.76       999
weighted avg       0.78      0.78      0.78       999

accuracy = 0.78
Epoch: 02 | Epoch Time: 5m 27s
	Train Loss: 0.529 | Train Acc: 75.60%
	 Val. Loss: 0.507 |  Val. Acc: 78.28%
              precision    recall  f1-score   support

           0       0.61      0.72      0.

# Load model and run on test data(GE18)

In [ ]:
del model
import gc
gc.collect()




379

In [ ]:

model = torch.load('HASOCOne_model_german_task_a_GermEval2019.pt')
model = model.to(device)

In [ ]:
#del model
#import gc
#gc.collect()

#model = torch.load('model_german_task_a.pt')
#model = model.to(device)

def evaluate_test():
    model.eval()
        
    total_eval_accuracy = 0
    total_eval_loss = 0
    number_of_eval_steps= 0
    
    all_true_labels = []
    all_pred_labels = []
    
    all_sentence_id=[]

    for batch in test_data_loader:
        b_sentence_id = batch[0].to(device)
        b_input_ids = batch[1].to(device)
        b_input_mask = batch[2].to(device)
        b_labels = batch[3].to(device)

        sent_ids = b_sentence_id.to('cpu').numpy()
        all_sentence_id.extend(sent_ids)
        
        with torch.no_grad():

            loss, logits = model(b_input_ids,
                                attention_mask= b_input_mask,
                                labels = b_labels.long(),
                                return_dict = False)
        total_eval_loss+=loss.item()

        logits = logits.detach().cpu().numpy()

        label_ids = b_labels.to('cpu').numpy()
        

        pred,true = predictions_labels(logits,label_ids)
        
        all_pred_labels.extend(pred)
        
        all_true_labels.extend(true)

    print(classification_report(all_pred_labels,all_true_labels))
    avg_val_accuracy = accuracy_score(all_pred_labels,all_true_labels)
    
    avg_val_loss = total_eval_loss/len(valid_data_loader)

    print("accuracy = {0:.2f}".format(avg_val_accuracy))
    
    return avg_val_loss,avg_val_accuracy,all_sentence_id,all_pred_labels

valid_loss,valid_acc,all_sentence_id,all_pred_labels = evaluate_test()



              precision    recall  f1-score   support

           0       0.50      0.64      0.56       941
           1       0.85      0.77      0.81      2591

    accuracy                           0.73      3532
   macro avg       0.68      0.70      0.68      3532
weighted avg       0.76      0.73      0.74      3532

accuracy = 0.73


In [ ]:
pred_labels = list(le.inverse_transform(all_pred_labels))

In [ ]:
from collections import OrderedDict
dict_index_y_value = {}

for index,y_value in zip(all_sentence_id,pred_labels):
    dict_index_y_value[index] = y_value

od = OrderedDict(sorted(dict_index_y_value.items()))

sorted_y_predicts = []
for k,v in od.items():
    sorted_y_predicts.append(v)



In [ ]:
df_test['index_col'] = df_test.index


In [ ]:
task = "task1"

In [ ]:
test_tweet_id = list(df_test['index_col'].values)
test_ID = list(df_test['index_col'].values)

test_final = pd.DataFrame(zip(test_tweet_id,sorted_y_predicts,test_ID),columns=['index_col',task,'index_col'])

if task == 'task1':
    test_final.to_csv("submission_DE_A_HASOCOne_german_task_a_GermEval2019_GermEval2018.csv.csv",index=False)

if task == 'task2':
    test_final.to_csv("submission_DE_B.csv",index=False)



In [ ]:
del model
import gc
gc.collect()



217